# K-NN on Amazon Fine Food Reviews dataset

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

Number of reviews: 568,454
Number of users: 256,059
Number of products: 74,258
Timespan: Oct 1999 - Oct 2012
Number of Attributes/Columns in data: 10

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review

#### Objective:
Classifying Amazon fine food reviews with polarity based color-coding (that means predicting wheater review is positive or not)

# Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently.
<br> 

In [1]:

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

con = sqlite3.connect('database.sqlite') #connecting to 'database.sqlite' and reading data from it 

filter_amz_data = pd.read_sql_query(""" 
SELECT *
FROM Reviews
WHERE Score != 3
""", con) # taking only those reviews having score greater than 3(4,5) and less than 3(1,2) not taking reviews of score 3


def partition_positive_negative(x): # Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filter_amz_data['Score']
positiveNegative = actualScore.map(partition_positive_negative) #map() will maps inputs of 'actualscore' using function partition
filter_amz_data['Score'] = positiveNegative

In [2]:
sample_amz_data = filter_amz_data.head(30000) #Sampled amazon fine foood reviews filtered data to 15k datapoints for time effiecieny

#  Exploratory Data Analysis

## [7.1.2] Data Cleaning: Deduplication


In [3]:
#Sorting data according to ProductId in ascending order; sorting is necessary because if duplicates entries are there than we want to have only one of it and that we get from first entry of duplicates entries after sorting

sorted_data = sample_amz_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

**sorting is necessary because if duplicates entries are there than we want to have only one of it and that we get from first entry of duplicates entries after sorting**

In [4]:
#Deduplication of entries

final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
print(sorted_data.shape) 
print(final.shape)

(30000, 10)
(28072, 10)


**As we can see that in sorted_data there are 15k points but now after removing duplicates entries we git some 14k points**

In [5]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
# It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator 
#which is not practically possible hence these two rows too are removed from calcualtions

In [6]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(28072, 10)


positive    23606
negative     4466
Name: Score, dtype: int64

##  Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [7]:
import re
import nltk
import string
from nltk.corpus import stopwords # nltk- natural language processing toolkit
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [8]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values: #taking each reviews
    filtered_sentence=[]
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split(): # taking each words of each reviews
        for cleaned_words in cleanpunc(w).split(): #cleanpunc(w) will remove any punctuations/special symbols from each word
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):   #checking word should be only have alphabets not aplha-numeric/numeric and all words should have length >2 
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8') #returing each word to be in lower case
                    filtered_sentence.append(s) # appending each filtered word of reviews to the list
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)
    i+=1

In [9]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

<b>Sorting dataset based on 'Time' feature</b>

In [10]:
sorted_Time_data = final.sort_values('Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')


# K-NN using 'Brute force'

<b><h1>Techniques for vectorization :--</h1> </b>

# (1) Bag of Words (BoW)

In [11]:
#Bow

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer() 
final_counts_Bow= count_vect.fit_transform(sorted_Time_data['CleanedText'].values) # computing Bow
print(type(final_counts_Bow))
print(final_counts_Bow.shape)

#Spliting dataset to training(with first 70% points) and test(remaining 30% points) dataset
n=28072 
tr=final_counts_Bow[0:(int(n*0.7))+1,:] #taking first 70% of points from sorted dataset based on 'Time' feature
labels_Bow_tr=sorted_Time_data['Score'].head(int(n*0.7)+1)


ts=final_counts_Bow[(int(n*0.7))+1:,:]   #taking remaining 30% of points from sorted dataset based on 'Time' feature
labels_Bow_ts=sorted_Time_data['Score'].tail(int(n*0.3))
print(tr.shape)
print(labels_Bow_tr.shape)
print(ts.shape)
print(labels_Bow_ts.shape)


# Data-preprocessing: Standardizing the data

from sklearn.preprocessing import MaxAbsScaler
standardized_data_train = MaxAbsScaler().fit_transform(tr)
standardized_data_test = MaxAbsScaler().fit_transform(ts)
print(standardized_data_train.shape)
print(standardized_data_test.shape)



<class 'scipy.sparse.csr.csr_matrix'>
(28072, 19423)
(19651, 19423)
(19651,)
(8421, 19423)
(8421,)
(19651, 19423)
(8421, 19423)


In [4]:
from scipy import stats
a=stats.uniform(2, 10)
type(a)

scipy.stats._distn_infrastructure.rv_frozen

In [12]:
#Calculating 10 fold CV 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

# creating odd list of K for KNN
myList = list(range(0,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k,algorithm='brute', n_jobs=-1)
    scores = cross_val_score(knn, standardized_data_train, labels_Bow_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)


C:\Users\Ashish\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)



The optimal number of neighbors is 5.


In [40]:
# ============================== KNN with k = optimal_k ===============================================
# instantiate learning model k = optimal_k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)

# fitting the model
knn_optimal.fit(standardized_data_train, labels_Bow_tr)

# predict the response
pred = knn_optimal.predict(standardized_data_test)
pred1 = knn_optimal.predict(standardized_data_train)


# evaluate accuracy
acc = accuracy_score(labels_Bow_ts, pred) * 100
acc1 = accuracy_score(labels_Bow_tr, pred1) * 100

print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc1))


The accuracy of the knn classifier for k = 5 is 83.493647%

The accuracy of the knn classifier for k = 5 is 86.972673%


# (2) TF-IDF 

In [35]:
#tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vect = TfidfVectorizer()
final_counts_tfidf= tf_idf_vect.fit_transform(sorted_Time_data['CleanedText'].values) 
print(type(final_counts_tfidf))
print(final_counts_tfidf.shape)

#Spliting dataset to training(with first 70% points) and test(remaining 30% points) dataset
n=len(sorted_Time_data)
tr_tfidf=final_counts_tfidf[0:(int(n*0.7))+1,:]  #taking first 70% of points from sorted dataset based on 'Time' feature
labels_tfidf_tr=sorted_Time_data['Score'].head(int(n*0.7)+1)

ts_tfidf=final_counts_tfidf[(int(n*0.7))+1:,:]   #taking remaining 30% of points from sorted dataset based on 'Time' feature
labels_tfidf_ts=sorted_Time_data['Score'].tail(int(n*0.3))
print(tr_tfidf.shape)
print(labels_tfidf_tr.shape)
print(ts_tfidf.shape)
print(labels_tfidf_ts.shape)


# Data-preprocessing: Standardizing the data

from sklearn.preprocessing import MaxAbsScaler
standardized_data_train_tfidf = MaxAbsScaler().fit_transform(tr_tfidf)
standardized_data_test_tfidf = MaxAbsScaler().fit_transform(ts_tfidf)
print(standardized_data_train_tfidf.shape)
print(standardized_data_test_tfidf.shape)




<class 'scipy.sparse.csr.csr_matrix'>
(28072, 19423)
(19651, 19423)
(19651,)
(8421, 19423)
(8421,)
(19651, 19423)
(8421, 19423)


In [36]:
#Calculating 10 fold CV 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

# creating odd list of K for KNN
myList_tfidf = list(range(0,50))
neighbors_tfidf = list(filter(lambda x: x % 2 != 0, myList_tfidf))

# empty list that will hold cv scores
cv_scores_tfidf = []

# perform 10-fold cross validation
for k in neighbors_tfidf:
    knn = KNeighborsClassifier(n_neighbors=k,algorithm='brute', n_jobs=-1)
    scores = cross_val_score(knn, standardized_data_train_tfidf, labels_tfidf_tr, cv=10, scoring='accuracy')
    cv_scores_tfidf.append(scores.mean())

# changing to misclassification error
MSE_tfidf = [1 - x for x in cv_scores_tfidf]

# determining best k
optimal_k_tfidf = neighbors_tfidf[MSE_tfidf.index(min(MSE_tfidf))]
print('\nThe optimal number of neighbors is %d.' % optimal_k_tfidf)



The optimal number of neighbors is 5.


In [35]:
# ============================== KNN with k = optimal_k ===============================================
# instantiate learning model k = optimal_k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k_tfidf)

# fitting the model
knn_optimal.fit(standardized_data_train_tfidf, labels_tfidf_tr)

# predict the response
pred_tfidf = knn_optimal.predict(standardized_data_test_tfidf)

# evaluate accuracy
acc_tfidf = accuracy_score(labels_tfidf_ts, pred_tfidf) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k_tfidf, acc_tfidf))


The accuracy of the knn classifier for k = 5 is 82.947393%


# (3) Average word2vec

In [20]:
# Train our own Word2Vec model using our own text corpus
import gensim
i=0
list_of_sent=[]
for sent in sorted_Time_data['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
    

In [21]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=1,size=50, workers=4)    
# min_count here says that if a word doesn't occur atleast 5 times than not construct its word2vector
# size means dimension of each vector of word  , here is 50 dimension

In [22]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)


In [23]:
#Spliting dataset to training(with first 70% points) and test(remaining 30% points) dataset

n=len(sent_vectors)
tr_avgw2v=sent_vectors[0:(int(n*0.7))+1]  #taking first 70% of points from sorted dataset based on 'Time' feature
labels_avgw2v_tr=sorted_Time_data['Score'].head(int(n*0.7)+1)

ts_avgw2v=sent_vectors[(int(n*0.7))+1:]  #taking remaining 30% of points from sorted dataset based on 'Time' feature
labels_avgw2v_ts=sorted_Time_data['Score'].tail(int(n*0.3))
print(len(tr_avgw2v))
print(labels_avgw2v_tr.shape)
print(len(ts_avgw2v))
print(labels_avgw2v_ts.shape)

# Data-preprocessing: Standardizing the data

from sklearn.preprocessing import MaxAbsScaler
standardized_data_train_avgw2v = MaxAbsScaler().fit_transform(tr_avgw2v)
standardized_data_test_avgw2v = MaxAbsScaler().fit_transform(ts_avgw2v)
print(standardized_data_train_avgw2v.shape)
print(standardized_data_test_avgw2v.shape)


19651
(19651,)
8421
(8421,)
(19651, 50)
(8421, 50)


In [25]:
#Calculating 10 fold CV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

# creating odd list of K for KNN
myList_avgw2v = list(range(0,50))
neighbors_avgw2v = list(filter(lambda x: x % 2 != 0, myList_avgw2v))

# empty list that will hold cv scores
cv_scores_avgw2v = []

# perform 10-fold cross validation
for k in neighbors_avgw2v:
    knn = KNeighborsClassifier(n_neighbors=k,algorithm='brute', n_jobs=-1)
    scores = cross_val_score(knn, standardized_data_train_avgw2v, labels_avgw2v_tr, cv=10, scoring='accuracy')
    cv_scores_avgw2v.append(scores.mean())

# changing to misclassification error
MSE_avgw2v = [1 - x for x in cv_scores_avgw2v]

# determining best k
optimal_k_avgw2v = neighbors_avgw2v[MSE_avgw2v.index(min(MSE_avgw2v))]
print('\nThe optimal number of neighbors is %d.' % optimal_k_avgw2v)



The optimal number of neighbors is 17.


In [26]:
# ============================== KNN with k = optimal_k ===============================================
# instantiate learning model k = optimal_k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k_avgw2v)

# fitting the model
knn_optimal.fit(standardized_data_train_avgw2v, labels_avgw2v_tr)

# predict the response
pred_avgw2v = knn_optimal.predict(standardized_data_test_avgw2v)

# evaluate accuracy
acc_avgw2v = accuracy_score(labels_avgw2v_ts, pred_avgw2v) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k_avgw2v, acc_avgw2v))


The accuracy of the knn classifier for k = 17 is 84.799905%


# (4) TF-IDF Weighted Word2Vec

In [28]:
#tf-idf weighted w2v

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfw2v_vect = TfidfVectorizer()
final_counts_tfidfw2v= tfidfw2v_vect.fit_transform(sorted_Time_data['Text'].values) 
print(type(final_counts_tfidfw2v))
print(final_counts_tfidfw2v.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(28072, 31375)


In [29]:
t=tfidfw2v_vect.get_feature_names()
t1=list()
ro=0
for sen in list_of_sent:
    revec=np.zeros(50)
    wrsu=0
    for wro in sen:
        try:
            wve=w2v_model.wv[wro]
            tfv=final_counts_tfidfw2v[ro,t.index(wro)]
            revec += (wve*tfv)
            wrsu += tfv
        except:
            pass
    revec /= wrsu
    t1.append(revec)
    ro += 1

In [31]:
#Spliting dataset to training(with first 70% points) and test(remaining 30% points) dataset

n=len(t1)
tr_tfidfw2v=t1[0:(int(n*0.7))+1]  #taking first 70% of points from sorted dataset based on 'Time' feature
labels_tfidfw2v_tr=sorted_Time_data['Score'].head(int(n*0.7)+1)

ts_tfidfw2v=t1[(int(n*0.7))+1:]  #taking remaining 30% of points from sorted dataset based on 'Time' feature
labels_tfidfw2v_ts=sorted_Time_data['Score'].tail(int(n*0.3))
print(len(tr_tfidfw2v))
print(labels_tfidfw2v_tr.shape)
print(len(ts_tfidfw2v))
print(labels_tfidfw2v_ts.shape)

# Data-preprocessing: Standardizing the data

from sklearn.preprocessing import MaxAbsScaler
standardized_data_train_tfidfw2v = MaxAbsScaler().fit_transform(tr_tfidfw2v)
standardized_data_test_tfidfw2v = MaxAbsScaler().fit_transform(ts_tfidfw2v)
print(standardized_data_train_tfidfw2v.shape)
print(standardized_data_test_tfidfw2v.shape)


19651
(19651,)
8421
(8421,)
(19651, 50)
(8421, 50)


In [32]:
# Calculating 10 fold CV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

# creating odd list of K for KNN
myList_tfidfw2v = list(range(0,50))
neighbors_tfidfw2v = list(filter(lambda x: x % 2 != 0, myList_tfidfw2v))

# empty list that will hold cv scores
cv_scores_tfidfw2v = []

# perform 10-fold cross validation
for k in neighbors_tfidfw2v:
    knn = KNeighborsClassifier(n_neighbors=k,algorithm='brute', n_jobs=-1)
    scores = cross_val_score(knn, standardized_data_train_tfidfw2v, labels_tfidfw2v_tr, cv=10, scoring='accuracy')
    cv_scores_tfidfw2v.append(scores.mean())

# changing to misclassification error
MSE_tfidfw2v = [1 - x for x in cv_scores_tfidfw2v]

# determining best k
optimal_k_tfidfw2v = neighbors_tfidfw2v[MSE_tfidfw2v.index(min(MSE_tfidfw2v))]
print('\nThe optimal number of neighbors is %d.' % optimal_k_tfidfw2v)



The optimal number of neighbors is 15.


In [33]:
# ============================== KNN with k = optimal_k ===============================================
# instantiate learning model k = optimal_k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k_tfidfw2v)

# fitting the model
knn_optimal.fit(standardized_data_train_tfidfw2v, labels_tfidfw2v_tr)

# predict the response
pred_tfidfw2v = knn_optimal.predict(standardized_data_test_tfidfw2v)

# evaluate accuracy
acc_tfidfw2v = accuracy_score(labels_tfidfw2v_ts, pred_tfidfw2v) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k_tfidfw2v, acc_tfidfw2v))


The accuracy of the knn classifier for k = 15 is 84.170526%


<b>Final Observation(s):--</b>

1. After all text-preprocesing step we have sorted the final dataset based on 'Time' features.

2. Than for each featurization or vectorization techniques we have first split the dataset into Training dataset(first 70% of points) than test dataset(remaining 30% of points).

3. Optimal 'K' and accuracy score for each techniques using 'Brute force' method are as follows:--

    For **Bow** the accuracy of the Knn classifier for **K=5** is **83.493647%**
    
    For **Tf-df** the accuracy of the Knn classifier for **K=5** is **82.947393%**
    
    For **Avgw2v** the accuracy of the Knn classifier for **K=17** is **84.799905%**
    
    For **Tfidfw2v** the accuracy of the Knn classifier for **K=15** is **84.170526%**

4. And found that Average word2vec perform better than other techniques.
            